In [20]:
import pandas as pd
import json

def preprocess_ncrb_data(file_path: str, output_file: str = 'crime_scores.json'):
    # 1. Load Excel
    try:
        df = pd.read_excel(file_path, header=2)
    except Exception as e:
        print(f"Error reading the Excel file: {e}")
        return

    # 2. Clean column names
    df.columns = df.columns.str.strip()

    # 3. Determine District column
    district_col_candidates = ['State/UT/District', 'State / UT / District', 'State/UT/District ', 'Unnamed: 0', 'Unnamed: 1']
    for col in district_col_candidates:
        if col in df.columns:
            df.rename(columns={col: 'District'}, inplace=True)
            break
    else:
        print("❌ Could not find a column for District.")
        print("Columns found:", df.columns.tolist())
        return

    # 4. Relevant crime columns (you can adjust according to your Excel)
    relevant_crimes = [
        'Murder (Sec.302 IPC)',
        'Attempt to Commit Murder (Sec.307 IPC)',
        'Rape (Sec. 376 IPC)',
        'Kidnapping and Abduction',
        'Robbery  (Sec.392/394/397 IPC)',
        'Theft (Section 379 IPC)',
        'Rioting (Sec.1470151 IPC)'
    ]

    # Keep only existing relevant columns
    existing_crimes = [col for col in relevant_crimes if col in df.columns]
    if not existing_crimes:
        print("❌ None of the relevant crime columns exist in this file.")
        print("Columns found:", df.columns.tolist())
        return

    # 5. Select relevant columns
    df_selected = df[['District'] + existing_crimes].copy()

    # 6. Clean data
    df_selected = df_selected[~df_selected['District'].str.contains('TOTAL', na=False)]
    df_selected = df_selected.dropna(subset=['District'])

    # Convert to numeric
    for col in existing_crimes:
        df_selected[col] = pd.to_numeric(df_selected[col], errors='coerce').fillna(0)

    # 7. Calculate scores
    df_selected['raw_crime_score'] = df_selected[existing_crimes].sum(axis=1)
    max_score = df_selected['raw_crime_score'].max()
    df_selected['normalized_score'] = (df_selected['raw_crime_score'] / max_score) * 10
    df_selected['normalized_score'] = df_selected['normalized_score'].round(2)

    # 8. Export JSON
    output_dict = pd.Series(df_selected.normalized_score.values, index=df_selected.District).to_dict()
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(output_dict, f, indent=4, ensure_ascii=False)

    print(f"✅ Successfully preprocessed the data and saved to {output_file}")
    print(f"Processed {len(output_dict)} districts.")

# --- RUN ---
if __name__ == "__main__":
    file_path = r"C:\Users\nagen\OneDrive\Documents\Desktop\jupyter projects\17016833111DistrictwiseIPCCrimes2022.xlsx"
    preprocess_ncrb_data(file_path)


✅ Successfully preprocessed the data and saved to crime_scores.json
Processed 114 districts.


In [11]:
r"C:\Users\nagen\OneDrive\Documents\Desktop\jupyter projects\17016833111DistrictwiseIPCCrimes2022.xlsx"

'C:\\Users\\nagen\\OneDrive\\Documents\\Desktop\\jupyter projects\\17016833111DistrictwiseIPCCrimes2022.xlsx'